**Notebook to demonstrate the reading and splitting of the hourly electricity data.**<br>

Comments about data structure, the splitting on the data into train, validation, and test sets, etc., are given below.

In [ ]:
# Use wget to download the data stored in csv format.
!wget "https://frankfurt-school-dataset.s3.eu-central-1.amazonaws.com/Electricity_data_hourly_products.csv"

--2020-09-30 14:42:44--  https://frankfurt-school-dataset.s3.eu-central-1.amazonaws.com/Electricity_data_hourly_products.csv
Resolving frankfurt-school-dataset.s3.eu-central-1.amazonaws.com (frankfurt-school-dataset.s3.eu-central-1.amazonaws.com)... 52.219.72.112
Connecting to frankfurt-school-dataset.s3.eu-central-1.amazonaws.com (frankfurt-school-dataset.s3.eu-central-1.amazonaws.com)|52.219.72.112|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 350029849 (334M) [text/csv]
Saving to: ‘Electricity_data_hourly_products.csv.1’

Electricity_data_ho 100%[===================>] 333.81M  28.0MB/s    in 13s     

2020-09-30 14:42:57 (26.0 MB/s) - ‘Electricity_data_hourly_products.csv.1’ saved [350029849/350029849]



In [ ]:
# List the directory; the downloaded file should be there. 
!ls -lh

total 668M
-rw-r--r-- 1 root root 334M Sep 30 08:49 Electricity_data_hourly_products.csv
-rw-r--r-- 1 root root 334M Sep 30 08:49 Electricity_data_hourly_products.csv.1
drwxr-xr-x 1 root root 4.0K Sep 28 16:35 sample_data


In [ ]:
# Load the data using the pandas library. Use the 1st (0th) column as index
import pandas as pd
df = pd.read_csv('Electricity_data_hourly_products.csv', index_col=0)

# Display the first 5 rows of the data; for a description of the content, see the text below this cell
display(df.head())

# Display basic dataframe info
df.info()

,contractId,qty,px
Datetime,,,
2020-03-01 11:59:13.229,11629792,0.5,-0.99
2020-03-01 11:59:13.243,11629792,0.5,-0.99
2020-03-01 11:59:13.243,11629792,0.1,-1.00
2020-03-01 11:59:46.669,11629792,3.0,-0.99
2020-03-01 11:59:55.065,11629792,3.0,-0.99


<class 'pandas.core.frame.DataFrame'>
Index: 8219996 entries, 2020-03-01 11:59:13.229 to 2020-08-24 20:42:39.432
Data columns (total 3 columns):
 #   Column      Dtype  
---  ------      -----  
 0   contractId  int64  
 1   qty         float64
 2   px          float64
dtypes: float64(2), int64(1)
memory usage: 250.9+ MB


**The content of the dataframe is the following:**<br>
Each row corresponds to a single electricity trade; the information given are all related to that.
* *Datetime*: the time that the electricity trade takes place. This is not the delivery time of the electricity, it is simply the time of the trade itself. This column is the index of the dataframe.<br>
* *contractId*: the id of the contract. The contractId identifies a given time window in which the electricity is to be provided (the "sell" leg) and used up (the "buy" leg). In this exercise, only hourly contracts, that is contracts which are about 1-hour-long time windows, are considered. There can be many trades under a given contractId. Trades, naturally, predate the time for which the electricity is to be actually delivered.
* *qty*: the quantity of the electricty being traded, given in MWH units.
* *px*: the price of 1 MWH of electricity, given in EUR.

Given all the above, it is clear that our data contains historical information about the price changes of electricity.

As each contractId in our data belongs to similar (hourly) contracts, all of them can be considered, there is no need to discard any of them. Each contract can be considered a time series of its own, pulled / sampled from the same population of price evolution over time. Data from different contracts, however, should not be mixed with each other; even if trades are happening at the same time for two different contracts, the prices can and will be different.





The prediction scneario is as follows. We are at the very beginning of an hour and we would like to know the price of electricity at the end of the current hour. We know all the trades that have just taken place the previous hour, while there are no trades yet in the current one. Can we then predict what the price will be by the end of the hour?

**Exercise steps:**
* Separately for all contracts, compute the OHLCV (Open, High, Low, Close, Volume) values with hourly binning for all trades. The volumes from the trades should be summed. Such OHCLV values will be your features.
  * A possible SOLUTION is something along the following lines:<br>
```
    def bin_ohlcv(df, contractId, binning_size='H'):
        df_cid = df[df.contractId == contractId]
        # resample for a binsize and the ohlc the result; and volume too.
        data = df_cid[['px']].resample(binning_size).ohlc().px
        data['volsum'] = df_cid[['qty']].resample(binning_size).sum()
        return data
```
* If no trades took place during a given hour, use the close price of the last hour to replace the nano OHLC values and use 0 for the nan volume.
  * A possible SOLUTION is something along the following lines:<br>
```
    def fillna_close(df):
        for i in df.iterrows():
            if not np.isnan(i[1]['close']):
                close = i[1]['close']
            else:
                i[1]['open'] = close
                i[1]['high'] = close
                i[1]['low'] = close
                i[1]['close'] = close
        return df
```

* Define how many hours of the immediate past should be considered as predictors for the next hour. Treating the contracts separately, prepare the OHLCV values of the past hours as features. Use a Segmenter for this.
  * A possible SOLUTION is something along the following lines (note: the code below sits within a contractId loop; X_train is already for a given contractId!):<br>
```
    # forecast defines how far in the future one predicts
    segmenter = SegmentXYForecast(width=window_size, step=1, y_func=last, forecast=1)

    if X_train.shape[0] < window_size+forecast_distance:
        error_cids[cid] = X_train.shape[0]
    else:
        # Selecting the y colum
        predict_column='close'
        y_train = X_train[predict_column]
        # Making a windowed version of the data with seglearn segmenter
        X_train_rolled, y_train_rolled, _= segmenter.fit_transform([X_train.values],[y_train.values.flatten()])
        # getting the segmented indices, aka dates
        X_train_index_rolled, _, _ = segmenter.fit_transform([X_train.index.values],[y_train.values.flatten()])
        count = 0
        # Iterating through the seglearn output windows (3D arrays)
        for i in X_train_rolled:
            # Flattening X_train_rolled to append as a row to the return df of df_windowing function
            data = pd.Series(i.flatten())
            # Appending flattened data
            train_df = train_df.append(data, ignore_index=True)
            # If getContractId is True append its value to the return df of df_windowing function
            if getContractId:
                train_df['contractId'].iloc[-1] = str(cid)
            # Appending y values from the y_train_rolled to the last added window row
            train_df['y'].iloc[-1] = y_train_rolled[count]
            count = count+1
```
At the end, for contract id 116330300 the data should look like this one (numerical values might be different for volume):
```
contractId,y,0,1,2,3,4,5,6,7,8,9
11630300,24.51,29.99,30.0,29.99,30.0,10.399999999999999,30.02,30.8,28.82,28.82,199.8
11630300,31.4,30.02,30.8,28.82,28.82,199.8,20.51,28.82,20.51,24.51,559.0000000000002
11630300,30.09,20.51,28.82,20.51,24.51,559.0000000000002,25.99,31.4,25.9,31.4,939.5999999999999
11630300,26.91,25.99,31.4,25.9,31.4,939.5999999999999,30.4,31.19,27.2,30.09,902.8000000000005
11630300,23.7,30.4,31.19,27.2,30.09,902.8000000000005,30.09,30.99,26.54,26.91,763.2
11630300,23.69,30.09,30.99,26.54,26.91,763.2,26.6,28.4,18.57,23.7,2491.800000000002
11630300,21.0,26.6,28.4,18.57,23.7,2491.800000000002,23.7,26.71,21.01,23.69,1894.0000000000005
11630300,23.41,23.7,26.71,21.01,23.69,1894.0000000000005,23.0,25.6,19.9,21.0,2427.2
```
Here one can see the contractId, the target y (the close price of the next hour), and the 2*5 OHLCV values (for a history length of 2 hours). It is clearly visible how the y values are indeed the close values of the next hour.
* Define your predictors (X) and the target (y). Don't use the contractId as a predictor! It was needed for the OHLCV computation and the segmentation, but it can be dropped now. Split the data into Train, Validation, and Test sets using train_test_split from sklearn. In order to make the results reproduceable, make sure that the "shuffle" parameter of the function is False. Look up the documentation and understand why this is needed. As the train_test_split returns only two data sets instead of the three we need, it is to be applied twice. Use 95% for Train and 5% for Test set during the first split. Then, for the second call, split off 5% from the just created Train set as Validation set.
  * A possible SOLUTION is something along the following lines:<br>
```
    X = train_df.drop(columns=['y','contractId'])
    y = train_df['y']

    # Creating train and test splits
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=split_size, shuffle=False)
    # Creating by splitting from remaning data of train
    X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=split_size, shuffle=False)
```
* Define models! Use the Train data to fit, and the Validation data to evaluate your models!